# Live Webcam Road Sign Detection

Real-time road sign detection using your webcam and the trained YOLOv8 model.

**Requirements**: Make sure you have a working webcam connected to your computer.

**Controls**:
- Press 'q' to quit
- Press 's' to save current frame
- Press 'r' to toggle recording

## Setup and Import Libraries

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import time
import os

print("Libraries imported successfully!")
print("OpenCV version:", cv2.__version__)

Libraries imported successfully!
OpenCV version: 4.12.0


## Load the Trained Model

In [2]:
model = YOLO("models/road_sign_detector_final.pt")
print("Road sign detection model loaded successfully!")

class_names = ['speedlimit', 'stop', 'trafficlight', 'crosswalk']
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]  # BGR colors for each class

print(f"Model can detect: {', '.join(class_names)}")

Road sign detection model loaded successfully!
Model can detect: speedlimit, stop, trafficlight, crosswalk


## Test Webcam Connection

In [4]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam")
    print("Make sure your webcam is connected and not being used by another application")
else:
    print("Webcam connected successfully!")
    
    ret, frame = cap.read()
    if ret:
        height, width = frame.shape[:2]
        print(f"Webcam resolution: {width}x{height}")
    
    cap.release()
    print("Webcam test completed.")

Webcam connected successfully!
Webcam resolution: 1920x1080
Webcam test completed.


## Simple Live Detection (Basic Version)

Basic webcam detection with simple display:

In [5]:
cap = cv2.VideoCapture(0)
frame_count = 0

print("Starting basic live detection...")
print("Press 'q' to quit")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        frame_count += 1
        
        # Run detection every few frames to improve performance
        if frame_count % 3 == 0:
            results = model(frame, verbose=False)
            
            # Draw detections
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        # Get box coordinates
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        confidence = float(box.conf[0])
                        class_id = int(box.cls[0])
                        
                        if confidence > 0.5:  # Only show confident detections
                            # Draw bounding box
                            color = colors[class_id]
                            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                            
                            # Draw label
                            label = f"{class_names[class_id]}: {confidence:.2f}"
                            cv2.putText(frame, label, (x1, y1-10), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Display frame
        cv2.imshow('Road Sign Detection - Press q to quit', frame)
        
        # Check for quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Detection interrupted by user")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print("Basic detection stopped.")

Starting basic live detection...
Press 'q' to quit
Basic detection stopped.


## Advanced Live Detection with Features

Enhanced version with FPS counter, save functionality, and better visualization:

In [ ]:
cap = cv2.VideoCapture(0)
frame_count = 0
start_time = time.time()
fps = 0
saved_count = 0

# Create results directory if it doesn't exist
os.makedirs("results/live_detection", exist_ok=True)

print("Starting advanced live detection...")
print("Controls:")
print("  'q' - Quit")
print("  's' - Save current frame with detections")
print("  'c' - Clear saved frame counter")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        
        frame_count += 1
        
        # Calculate FPS
        if frame_count % 30 == 0:
            end_time = time.time()
            fps = 30 / (end_time - start_time)
            start_time = end_time
        
        # Run detection
        if frame_count % 2 == 0:  # Process every 2nd frame for better performance
            results = model(frame, verbose=False)
            detection_count = 0
            
            # Draw detections
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        confidence = float(box.conf[0])
                        class_id = int(box.cls[0])
                        
                        if confidence > 0.3:  # Lower threshold for live detection
                            detection_count += 1
                            
                            # Draw bounding box with thickness based on confidence
                            thickness = int(3 * confidence)
                            color = colors[class_id]
                            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
                            
                            # Draw label with background
                            label = f"{class_names[class_id]}: {confidence:.2f}"
                            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                            
                            # Background rectangle for text
                            cv2.rectangle(frame, (x1, y1-text_height-10), (x1+text_width, y1), color, -1)
                            cv2.putText(frame, label, (x1, y1-5), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Draw info overlay
        info_text = f"FPS: {fps:.1f} | Detections: {detection_count if 'detection_count' in locals() else 0} | Saved: {saved_count}"
        cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Draw instructions
        cv2.putText(frame, "Press 'q' to quit, 's' to save", (10, frame.shape[0]-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Display frame
        cv2.imshow('Advanced Road Sign Detection', frame)
        
        # Handle key presses
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            saved_count += 1
            save_path = f"results/live_detection/detection_{saved_count:03d}.jpg"
            cv2.imwrite(save_path, frame)
            print(f"Frame saved: {save_path}")
        elif key == ord('c'):
            saved_count = 0
            print("Saved frame counter reset")

except KeyboardInterrupt:
    print("Detection interrupted by user")

finally:
    cap.release()
    cv2.destroyAllWindows()
    print(f"Advanced detection stopped. Total frames saved: {saved_count}")

Starting advanced live detection...
Controls:
  'q' - Quit
  's' - Save current frame with detections
  'c' - Clear saved frame counter


## Video File Detection

Process a video file instead of live webcam:

In [ ]:
video_path = "your_video.mp4"  # Replace with your video file path

if os.path.exists(video_path):
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video: {width}x{height} at {fps} FPS, {total_frames} frames")
    
    # Optional: Save output video
    output_path = "results/processed_video.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_num = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_num += 1
            
            # Progress indicator
            if frame_num % 30 == 0:
                progress = (frame_num / total_frames) * 100
                print(f"Processing: {progress:.1f}% ({frame_num}/{total_frames})")
            
            # Run detection
            results = model(frame, verbose=False)
            
            # Draw detections
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        confidence = float(box.conf[0])
                        class_id = int(box.cls[0])
                        
                        if confidence > 0.5:
                            color = colors[class_id]
                            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                            
                            label = f"{class_names[class_id]}: {confidence:.2f}"
                            cv2.putText(frame, label, (x1, y1-10), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Write frame to output video
            out.write(frame)
            
            # Display frame (optional - comment out for faster processing)
            cv2.imshow('Video Processing', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    except KeyboardInterrupt:
        print("Video processing interrupted")
    
    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"Video processing complete. Output saved to: {output_path}")

else:
    print(f"Video file not found: {video_path}")
    print("Update the video_path variable with your video file location")

## Troubleshooting

If you encounter issues:

### Webcam Not Working:
- Make sure no other applications are using the webcam
- Try different camera indices: `cv2.VideoCapture(1)` or `cv2.VideoCapture(2)`
- Check camera permissions in System Preferences (macOS) or Settings (Windows)

### Performance Issues:
- Increase frame skipping: change `frame_count % 3 == 0` to `frame_count % 5 == 0`
- Reduce video resolution
- Lower confidence threshold

### OpenCV Issues:
```bash
pip install opencv-python
# or if you have conda:
conda install opencv
```